In [5]:
import urllib.request
import zipfile
import glob
import re
import xml.etree.ElementTree as ET
import pandas as pd
import torch
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import datasets

from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration
from transformers.optimization import  Adafactor
from datasets import load_dataset

In [15]:
dataset = load_dataset('web_nlg', 'webnlg_challenge_2017')
metric = datasets.load_metric('sacrebleu')
o_triple_set = dataset['train']['original_triple_sets']

print(len(dataset))
#print(dataset)

"""for text in o_triple_set['otriple_set']:
    print(text[0])
    model.eval()
    input_ids = tokenizer.encode("WebNLG:{} </s>".format(text[0]), return_tensors="pt")  # Batch size 1
    print(input_ids)
    outputs = model.generate(input_ids)

    gen_text=tokenizer.decode(outputs[0]).replace('<pad>','').replace('</s>','')

    print(gen_text)"""

Reusing dataset web_nlg (/Users/furkansimsek/.cache/huggingface/datasets/web_nlg/webnlg_challenge_2017/0.0.0/28ffb892f7f42450dd9558684aa43bcaf44b1b3bf0d77cb8d73534646af88dda)


  0%|          | 0/3 [00:00<?, ?it/s]

3


'for text in o_triple_set[\'otriple_set\']:\n    print(text[0])\n    model.eval()\n    input_ids = tokenizer.encode("WebNLG:{} </s>".format(text[0]), return_tensors="pt")  # Batch size 1\n    print(input_ids)\n    outputs = model.generate(input_ids)\n\n    gen_text=tokenizer.decode(outputs[0]).replace(\'<pad>\',\'\').replace(\'</s>\',\'\')\n\n    print(gen_text)'

In [21]:
train_data = pd.read_csv('webNLG2020_train.csv', index_col=[0])

print('Number of rows: ' +  str(len(train_data)))

# dropping the last 205 rows so that one batch would not leave any remainder
train_data = train_data.iloc[:35000,:]

# return random samples of items in order to shuffle the data
train_data = train_data.sample(frac = 1)

batch_size = 8
epochs = 1

number_of_batches = int(len(train_data)/batch_size)
print('Number of training batches: ' + str(number_of_batches))

Number of rows: 35205
Number of training batches: 4375


In [22]:
# Checking for GPU availability
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the CPU


In [23]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

model.to(device=device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [24]:
optimizer = Adafactor(
    model.parameters(),
    lr=1e-3,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False
)

In [26]:
#Sets the module in training mode
model.train()

loss_per_10_steps=[]
for epoch in range(1,number_of_batches+1):
  print('Running epoch: {}'.format(epoch))

  running_loss=0

  for i in range(number_of_batches):
    inputbatch=[]
    labelbatch=[]
    new_df=train_data[i*batch_size:i*batch_size+batch_size]
    for indx,row in new_df.iterrows():
      input = 'WebNLG: '+row['input_text']+'</s>'
      labels = row['target_text']+'</s>'
      inputbatch.append(input)
      labelbatch.append(labels)
    inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=400,return_tensors='pt')["input_ids"]
    labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=400,return_tensors="pt") ["input_ids"]
    inputbatch=inputbatch.to(device)
    labelbatch=labelbatch.to(device)

    # clear out the gradients of all Variables
    optimizer.zero_grad()

    # Forward propogation
    outputs = model(input_ids=inputbatch, labels=labelbatch)
    loss = outputs.loss
    loss_num=loss.item()
    logits = outputs.logits
    running_loss+=loss_num
    if i%10 ==0:
      loss_per_10_steps.append(loss_num)

    # calculating the gradients
    loss.backward()

    #updating the params
    optimizer.step()

  running_loss=running_loss/int(number_of_batches)
  print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))


Running epoch: 1


/Users/furkansimsek/miniconda3/envs/prompt-tuning-for-NLG/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2228: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/Users/furkansimsek/miniconda3/envs/prompt-tuning-for-NLG/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:190: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
steps = [i*100 for i in range(len(loss_per_10_steps))]

plt.plot(steps, loss_per_10_steps)
plt.title('Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.show()

In [ ]:
model.eval()
input_ids = tokenizer.encode("WebNLG: sidharth | hometown | Delhi && sidharth | play |  football </s>", return_tensors="pt")  # Batch size 1
input_ids=input_ids.to(device)
outputs = model.generate(input_ids)tokenizer.decode(outputs[0])